In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from arguments import parse_args
from main import train, get_data
from dataset import LocalizationDataset
from torch.utils.data import DataLoader
import os
from cust_model import Tracker, TrackingDataset
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [3]:
from import_data import grab_amazon_data
inputs_prebatch, outputs_prebatch = grab_amazon_data()

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
args = AttrDict({"num_trajs":807,
        "sl":96,
        "batch_size":128,
        "num_particles":30,
        "window_length":6,
        "h":64,
        "emb_obs":23,
        "emb_act":32,
        "dropout":0.1, #we changed this from 0.5
        "obs_num":5,
        "resamp_alpha":0.5,
        "bp_length":10, 
        "clip":1.0,
        "lr":0.0005,
        "bpdecay":0.1,
        "l2_weight":1.0,
        "l1_weight":0.0})

In [ ]:
from cust_rnn import SimpleRNN

In [ ]:
train_dataset = TrackingDataset(inputs_prebatch, outputs_prebatch)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size,
                            num_workers=8, pin_memory=True, shuffle=False, drop_last = True)
# model = Tracker(args)
model = SimpleRNN(args)
optimizer = torch.optim.RMSprop(model.parameters(), lr=args.lr)

In [ ]:
cnt = 0
best_eval = 1000
from tqdm import tqdm

losses = []

print("starting epochs")
for epoch in tqdm(range(10)):
    per_e_loss = []
    # print("going to train")
    model.train() #just a toggle switch

    # print("starting iterations")
    for iteration, data in enumerate(train_loader):
        cnt = cnt + 1

        output, window = data
        model.zero_grad()
        loss, log_loss, particle_pred = model.step(
            window, output, args)
        loss.backward()

        if args.clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()

        loss_last = log_loss.to('cpu').detach().numpy()
        loss_all = loss.to('cpu').detach().numpy()
        per_e_loss.append([loss_last, loss_all])
    losses.append(per_e_loss)


starting epochs


  0%|          | 0/10 [00:00<?, ?it/s]/Users/michaelsprintson/Documents/nosync.nosync/pfrnns/cust_rnn.py:96: UserWarning: Using a target size (torch.Size([96, 128])) that is different to the input size (torch.Size([1, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  l2_pred_loss = torch.nn.functional.mse_loss(pred, gt_normalized, reduction='none') * bpdecay_params
/Users/michaelsprintson/Documents/nosync.nosync/pfrnns/cust_rnn.py:97: UserWarning: Using a target size (torch.Size([96, 128])) that is different to the input size (torch.Size([1, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  l1_pred_loss = torch.nn.functional.l1_loss(pred, gt_normalized, reduction='none') * bpdecay_params
/Users/michaelsprintson/Documents/nosync.nosync/pfrnns/cust_rnn.py:127: UserWarning: Using a target size (torch.Size([96])) that is different to the input size (torch.Size([1]))

In [ ]:
fig, (ax0,ax1) = plt.subplots(1,2,figsize = (10,5))
for lidx, lnp in enumerate(losses):
    lnp = np.array(lnp)
    ax0.plot(lnp[:,0], label = f"{lidx}_loss")
    ax1.plot(lnp[:,1], label = f"{lidx}_loss_last")